In [59]:
import pandas as pd
import os
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
from scipy.spatial.distance import squareform
import numpy as np
import matplotlib.pyplot as plt
from nilearn import datasets

In [60]:
def time_series_loader(root):
    ts_list = sorted(os.listdir(root))
    ts_path_list = []
    for i in range(0, len(ts_list)):
            ts_path_list.append(os.path.join(root, ts_list[i]))
    return ts_path_list

def atlas_labels():
    atlas = datasets.fetch_atlas_aal()
    labels = atlas.labels
    return labels

def save_hypergraph(hypergraph,directory,method,threshold,id):
    dir = f'{directory}/{method}/thresh_{threshold}'
    if not os.path.exists(dir):
        os.makedirs(dir)
    np.savetxt(f'{dir}/{id}_{method}_{threshold}.csv', hypergraph, delimiter=',')
    return


In [61]:
labels = atlas_labels()
nrois = len(labels)
time_series_list = time_series_loader('ADNI_gsr/hypergraphs/fourier')
threshold = 0.6


for i in range(len(time_series_list)):
    #id = time_series_list[i][-14:-4]
    id = time_series_list[i][-22:-12]
    time_series = np.loadtxt(time_series_list[i], delimiter=',')
    df = pd.DataFrame(time_series, columns=labels)
    corr = df.corr().values
    dissimilarity = 1 - abs(corr)
    dissimilarity[np.isnan(dissimilarity)] = 0
    Z = linkage(squareform(dissimilarity), 'complete')
    cluster_idx = fcluster(Z, threshold, criterion='distance')
    hg = np.zeros((nrois,nrois))
    for j in range(nrois):
        for k in range(nrois):
            if cluster_idx[j] == cluster_idx[k]:
                hg[j , k] = 1
            else:
                pass

    save_hypergraph(hg,'ADNI_gsr/hypergraphs','fourier_cluster',threshold,id)

In [ ]:
#plt.figure(figsize=(12,5))
#dendrogram(Z, labels=labels,orientation='top',
               #leaf_rotation=90);